In [8]:
import math
import random

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import editdistance
from transer import Dataset
from transer import Trainer
from IPython.display import clear_output
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
dataset = Dataset('train.csv')
words_vocab = dataset.words_vocab
trans_vocab = dataset.trans_vocab

In [17]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size, pad_idx):
        super(Encoder, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=pad_idx)
        self.gru       = nn.GRU(emb_size, hidden_size, batch_first = True)
    
    def forward(self, source, source_lens=None):
        embedded = self.embedding(source)
        
        if source_lens is not None:
            embedded = pack(embedded, source_lens, batch_first = True)
  
        outs, hidden = self.gru(embedded)
    
        if source_lens is not None:
            outs,_ = unpack(outs, batch_first = True)
        
        return outs, hidden

In [18]:
batch_words, batch_trans_in, batch_trans_out, words_lens, trans_lens = dataset.get_batch(32, sort = True)
encoder = Encoder(len(words_vocab), 32, 64, pad_idx = words_vocab.pad_idx)
outputs, hidden = encoder(batch_words) 
print(outputs.size(), hidden.size())

torch.Size([32, 14, 64]) torch.Size([1, 32, 64])
